In [1]:
import os
import numpy as np
import pandas as pd
import mne
from mne.decoding import CSP
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, DepthwiseConv2D, BatchNormalization, 
                                     Activation, AveragePooling2D, Dropout, Flatten, Dense)
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import matplotlib.pyplot as plt

# ==============================================================================
# CONFIGURATION
# ==============================================================================
MTC_BASE_PATH = '/kaggle/input/mtcaic3'
FINAL_Z_SCORE_THRESHOLD = 5.0
MOTION_THRESHOLD = 7.7966 
EEG_CHANNELS = ['FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8']
SFREQ = 250  # Sampling frequency
L_FREQ = 8.0 # Bandpass low cutoff
H_FREQ = 30.0 # Bandpass high cutoff

# ==============================================================================
# MODIFIED PREPROCESSING PIPELINE (WITH LABEL TRACKING)
# ==============================================================================
def load_mtc_mi_trial_data(row, base_path):
    id_num = row['id']
    if 4800 < id_num <= 4900: 
        split = 'validation'
    elif id_num > 4900: 
        split = 'test'
    else: 
        split = 'train'
        
    eeg_path = os.path.join(
        base_path, 'MI', split, row['subject_id'], 
        str(row['trial_session']), 'EEGdata.csv'
    )
    if not os.path.exists(eeg_path):
        return None
        
    eeg_data = pd.read_csv(eeg_path)
    trial_num = int(row['trial'])
    samples_per_trial = 2250
    start_idx = (trial_num - 1) * samples_per_trial
    end_idx = start_idx + samples_per_trial
    return eeg_data.iloc[start_idx:end_idx]

def check_for_motion_artifact(trial_df, threshold):
    motion_sensors = ['AccX', 'AccY', 'AccZ', 'Gyro1', 'Gyro2', 'Gyro3']
    if trial_df[motion_sensors].std().max() > threshold:
        return True
    return False

def check_for_zscore_artifact(trial_df, z_threshold):
    for channel in EEG_CHANNELS:
        if channel in trial_df.columns:
            data = trial_df[channel].values
            if np.any(np.abs(data) > z_threshold):
                return True
    return False

def run_ica_on_subject(subject_df):
    info = mne.create_info(
        ch_names=EEG_CHANNELS, 
        sfreq=SFREQ, 
        ch_types='eeg'
    )
    raw = mne.io.RawArray(
        subject_df[EEG_CHANNELS].T.values * 1e-6, 
        info
    )
    raw.filter(l_freq=1.0, h_freq=40.0, fir_design='firwin')
    ica = mne.preprocessing.ICA(
        n_components=8, 
        random_state=97, 
        max_iter='auto'
    )
    ica.fit(raw)
    eog_indices, _ = ica.find_bads_eog(raw, ch_name='FZ')
    ica.exclude = eog_indices
    ica.apply(raw)
    cleaned_df = raw.to_data_frame() * 1e6
    return cleaned_df

def run_preprocessing():
    print("Starting preprocessing pipeline...")
    # Load index files
    train_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'train.csv'))
    train_df['split'] = 'train'
    validation_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'validation.csv'))
    validation_df['split'] = 'validation'
    test_df = pd.read_csv(os.path.join(MTC_BASE_PATH, 'test.csv'))
    test_df['split'] = 'test'

    # Combine and filter MI trials
    all_trials_index = pd.concat([train_df, validation_df, test_df], ignore_index=True)
    mi_trials_index = all_trials_index[all_trials_index['task'] == 'MI'].copy()
    
    # Step 1: Motion artifact removal
    print("Phase 1: Removing motion artifacts...")
    motion_artifact_ids = []
    for idx, row in mi_trials_index.iterrows():
        raw_trial = load_mtc_mi_trial_data(row, MTC_BASE_PATH)
        if raw_trial is not None and check_for_motion_artifact(raw_trial, MOTION_THRESHOLD):
            motion_artifact_ids.append(row['id'])
    motion_clean_index = mi_trials_index[~mi_trials_index['id'].isin(motion_artifact_ids)]
    print(f"Removed {len(motion_artifact_ids)} trials with motion artifacts")

    # Step 2: Per-subject processing with label tracking
    print("Phase 2: Per-subject standardization and ICA...")
    grouped_data = {split: {} for split in ["train", "validation", "test"]}
    for idx, row in motion_clean_index.iterrows():
        trial_data = load_mtc_mi_trial_data(row, MTC_BASE_PATH)
        if trial_data is not None:
            split = row['split']
            subject = row['subject_id']
            label = row.get('label', None)
            trial_id = row['id']
            
            if subject not in grouped_data[split]:
                grouped_data[split][subject] = []
                
            grouped_data[split][subject].append({
                'data': trial_data,
                'label': label,
                'id': trial_id
            })

    # Process each subject
    cleaned_data = {'train': [], 'validation': [], 'test': []}
    for split in ["train", "validation", "test"]:
        print(f"  Processing {split} set...")
        for subject, trials in grouped_data[split].items():
            if not trials:
                continue
                
            # Concatenate all trials for the subject
            subject_df = pd.concat([t['data'] for t in trials], ignore_index=True)
            
            # Standardize
            subject_means = subject_df.mean()
            subject_stds = subject_df.std()
            subject_stds[subject_stds < 1e-8] = 1.0
            std_subject_df = (subject_df - subject_means) / subject_stds
            
            # Apply ICA
            ica_cleaned_subject_df = run_ica_on_subject(std_subject_df)
            
            # Split back into trials and store with metadata
            num_trials = len(trials)
            for i in range(num_trials):
                start = i * 2250
                end = start + 2250
                trial_data = ica_cleaned_subject_df.iloc[start:end]
                cleaned_data[split].append({
                    'data': trial_data,
                    'label': trials[i]['label'],
                    'id': trials[i]['id']
                })

    # Step 3: Final z-score rejection
    print("Phase 3: Final z-score rejection...")
    final_clean_data = {'train': [], 'validation': [], 'test': []}
    for split in ["train", "validation", "test"]:
        for trial in cleaned_data[split]:
            std_trial = (trial['data'] - trial['data'].mean()) / trial['data'].std()
            if not check_for_zscore_artifact(std_trial, FINAL_Z_SCORE_THRESHOLD):
                final_clean_data[split].append(trial)
    
    # Report final counts
    for split in ["train", "validation", "test"]:
        print(f"{split.capitalize()} set: {len(final_clean_data[split])} clean trials")
    
    return final_clean_data

# ==============================================================================
# EEGNET MODEL ARCHITECTURE
# ==============================================================================
def EEGNet(nb_classes, Chans=8, Samples=2250, 
           dropoutRate=0.5, kernLength=125, F1=8, D=2, F2=16, 
           dropoutType='Dropout'):
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D or Dropout')
    
    input1 = Input(shape=(Chans, Samples, 1))
    
    # Block 1
    block1 = Conv2D(F1, (1, kernLength), padding='same',
                   input_shape=(Chans, Samples, 1),
                   use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1),
                            depth_multiplier=D,
                            depthwise_constraint=max_norm(1.))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('elu')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropoutType(dropoutRate)(block1)
    
    # Block 2
    block2 = Conv2D(F2, (1, 16),
                   padding='same',
                   use_bias=False)(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('elu')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropoutType(dropoutRate)(block2)
    
    # Flatten and classify
    flatten = Flatten(name='flatten')(block2)
    dense = Dense(nb_classes, name='dense', 
                 kernel_constraint=max_norm(0.5))(flatten)
    softmax = Activation('softmax', name='softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)

# ==============================================================================
# DATA PREPARATION AND MODEL TRAINING
# ==============================================================================
def prepare_data_for_csp(trial_data):
    """Convert trial data to numpy array in (trials, channels, time) format"""
    return np.array([t['data'][EEG_CHANNELS].values.T for t in trial_data])

def apply_bandpass_filter(data, l_freq, h_freq):
    """Apply bandpass filter to EEG data"""
    info = mne.create_info(ch_names=EEG_CHANNELS, sfreq=SFREQ, ch_types='eeg')
    epochs = mne.EpochsArray(data, info)
    epochs.filter(l_freq, h_freq, method='iir', verbose=False)
    return epochs.get_data()

def main():
    # Step 1: Run preprocessing pipeline
    preprocessed_data = run_preprocessing()
    
    # Prepare training data
    train_trials = preprocessed_data['train']
    X_train = prepare_data_for_csp(train_trials)
    y_train = [t['label'] for t in train_trials]
    
    # Prepare validation data
    val_trials = preprocessed_data['validation']
    X_val = prepare_data_for_csp(val_trials)
    y_val = [t['label'] for t in val_trials]
    
    # Encode labels
    le = LabelEncoder()
    y_train_int = le.fit_transform(y_train)
    y_val_int = le.transform(y_val)
    
    # Apply bandpass filtering
    print("\nApplying bandpass filter (8-30 Hz)...")
    X_train_filt = apply_bandpass_filter(X_train, L_FREQ, H_FREQ)
    X_val_filt = apply_bandpass_filter(X_val, L_FREQ, H_FREQ)
    
    # Train CSP
    print("Training CSP with 8 components...")
    csp = CSP(n_components=8, reg=None, log=True, norm_trace=False)
    csp.fit(X_train_filt, y_train_int)
    
    # Transform data using CSP
    print("Applying CSP transformation...")
    X_train_csp = csp.transform(X_train_filt)
    X_val_csp = csp.transform(X_val_filt)
    
    # Reshape for EEGNet (add channel dimension)
    X_train_final = X_train_csp.reshape(X_train_csp.shape[0], 8, 2250, 1)
    X_val_final = X_val_csp.reshape(X_val_csp.shape[0], 8, 2250, 1)
    
    # Convert labels to categorical
    y_train_cat = to_categorical(y_train_int)
    y_val_cat = to_categorical(y_val_int)
    
    # Build and compile EEGNet
    print("\nBuilding EEGNet model...")
    model = EEGNet(nb_classes=2, Chans=8, Samples=2250, 
                  dropoutRate=0.5, kernLength=125, 
                  F1=8, D=2, F2=16)
    
    model.compile(loss='categorical_crossentropy', 
                 optimizer=Adam(learning_rate=0.001), 
                 metrics=['accuracy'])
    
    # Callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                  patience=5, min_lr=1e-6, verbose=1)
    early_stop = EarlyStopping(monitor='val_loss', patience=10, 
                              restore_best_weights=True, verbose=1)
    
    # Train model
    print("Training EEGNet model...")
    history = model.fit(
        X_train_final, y_train_cat,
        validation_data=(X_val_final, y_val_cat),
        epochs=100,
        batch_size=32,
        callbacks=[reduce_lr, early_stop],
        verbose=1
    )
    
    # Plot training history
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()
    
    # Prepare test data
    test_trials = preprocessed_data['test']
    X_test = prepare_data_for_csp(test_trials)
    test_ids = [t['id'] for t in test_trials]
    
    # Process test data
    X_test_filt = apply_bandpass_filter(X_test, L_FREQ, H_FREQ)
    X_test_csp = csp.transform(X_test_filt)
    X_test_final = X_test_csp.reshape(X_test_csp.shape[0], 8, 2250, 1)
    
    # Generate predictions
    print("\nGenerating test predictions...")
    test_probs = model.predict(X_test_final)
    test_preds = np.argmax(test_probs, axis=1)
    test_labels = le.inverse_transform(test_preds)
    
    # Create submission file
    submission = pd.DataFrame({
        'id': test_ids,
        'label': test_labels
    })
    submission.to_csv('submission.csv', index=False)
    print("Submission file created: submission.csv")
    
    # Validate model performance
    val_probs = model.predict(X_val_final)
    val_preds = np.argmax(val_probs, axis=1)
    val_accuracy = np.mean(val_preds == y_val_int)
    print(f"\nValidation Accuracy: {val_accuracy:.4f}")

if __name__ == '__main__':
    main()

2025-07-14 21:57:27.913194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752530248.159969      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752530248.232583      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Starting preprocessing pipeline...
Phase 1: Removing motion artifacts...
Removed 125 trials with motion artifacts
Phase 2: Per-subject standardization and ICA...
  Processing train set...
Creating RawArray with float64 data, n_channels=8, n_times=175500
    Range : 0 ... 175499 =      0.000 ...   701.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)

Fitting ICA to data using 8 channels (please be patient, this may take a while)


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished


Selecting by number: 8 components
Fitting ICA took 0.9s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition b

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


Fitting ICA took 0.2s.
Using EOG channel: FZ
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 2500 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff f

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


Train set: 705 clean trials
Validation set: 16 clean trials
Test set: 10 clean trials

Applying bandpass filter (8-30 Hz)...
Not setting metadata
705 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
16 matching events found
No baseline correction applied
0 projection items activated
Training CSP with 8 components...
Computing rank from data with rank=None
    Using tolerance 9.8e+04 (2.2e-16 eps * 8 dim * 5.5e+19  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Applying CSP transformation...


ValueError: cannot reshape array of size 5640 into shape (705,8,2250,1)